In [ ]:
def pedido(limit,page):
    url = f"https://api.itjobs.pt/job/list.json?api_key=ee176fa9456283ab9c42f357b036e236&limit={limit}&page={page}"
    payload = {}
    headers = {'User-Agent': "ALPCD_5"}
    res = requests.request("GET", url, headers=headers, data=payload)
    if res.status_code == 200:
        results = res.json()
        return results
    else:
        print(f"Erro {res.status_code}- {res.text}")
        return {}


limit = 100
page = 1
response = pedido(limit, page)
list_results= response["results"]

while page*limit < response["total"]:
    page += 1
    new_results = pedido(limit, page)["results"]
    list_results += new_results
response["results"] = list_results

1261


In [ ]:
sorted_results = sorted(list_results, key=lambda x: x["publishedAt"], reverse= True)
for sort in sorted_results:
    print(sort["publishedAt"])


2024-10-31 12:47:25
2024-10-31 12:47:02
2024-10-31 12:46:57
2024-10-31 12:46:53
2024-10-31 12:46:38
2024-10-31 12:46:34
2024-10-31 12:46:31
2024-10-31 12:46:24
2024-10-31 12:45:58
2024-10-31 12:45:55
2024-10-31 12:45:51
2024-10-31 12:45:46
2024-10-31 12:45:43
2024-10-31 12:45:39
2024-10-31 12:45:21
2024-10-31 12:45:17
2024-10-31 12:45:10
2024-10-31 12:45:07
2024-10-31 12:44:51
2024-10-31 12:44:46
2024-10-31 12:44:41
2024-10-31 12:44:32
2024-10-31 12:44:26
2024-10-31 12:44:12
2024-10-31 12:44:00
2024-10-31 12:43:59
2024-10-31 12:43:49
2024-10-31 12:43:47
2024-10-31 12:43:37
2024-10-31 12:42:58
2024-10-31 12:42:45
2024-10-30 19:09:38
2024-10-30 19:09:20
2024-10-30 19:09:13
2024-10-30 19:08:50
2024-10-30 19:08:40
2024-10-30 19:08:33
2024-10-30 19:08:29
2024-10-30 19:08:21
2024-10-30 19:08:18
2024-10-30 19:08:08
2024-10-30 19:08:03
2024-10-30 19:07:59
2024-10-30 19:07:54
2024-10-30 19:07:44
2024-10-30 19:07:41
2024-10-30 19:07:22
2024-10-30 19:07:18
2024-10-30 19:07:14
2024-10-30 19:07:05


In [51]:
lista = [1,2,3,4,5]
lista2 = [5,6,7]

lista += lista2
print(lista[None])

TypeError: list indices must be integers or slices, not NoneType